<a href="https://colab.research.google.com/github/komaltp/project/blob/main/scrap_email.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from bs4 import BeautifulSoup
import requests
import requests.exceptions
import urllib.parse
from collections import deque
import re
import pandas as pd
from google.colab import files


user_url = str(input('[+] Enter target URL to scan: '))
urls = deque([user_url])

scraped_urls = set()
emails = set()

count = 0
try:
  while len(urls):
    count += 1
    if count == 200:
      break
    url = urls.popleft()
    scraped_urls.add(url)

    parts = urllib.parse.urlsplit(url)
    base_url = '{0.scheme}://{0.netloc}'.format(parts)

    path = url[:url.rfind('/')+1] if '/' in parts.path else url

    print('[%d] Processing %s' % (count, url))
    try:
      response = requests.get(url)
    except (requests.exceptions.MissingSchema, requests.exceptions.ConnectionError):
      continue

    new_emails = set(re.findall(r'[a-z0-9\.\-+_]+@[a-z0-9\.\-+_]+\.[a-z]+', response.text, re.I))
    emails.update(new_emails)
    #print(emails)
    soup = BeautifulSoup(response.text, features="lxml")

    for anchor in soup.find_all("a"):
      link = anchor.attrs['href'] if 'href' in anchor.attrs else ''
      if link.startswith('/'):
        link = base_url + link
      elif not link.startswith('http'):
        link = path + link
      if not link in urls and not link in scraped_urls:
        urls.append(link)
except KeyboardInterrupt:
  print('[-] Closing!')

for mail in emails:
  print(mail)
df = pd.DataFrame(emails, columns=["Email"])
df.to_csv('email.csv', index=False)

files.download("email.csv")

[+] Enter target URL to scan: https://www.amazon.de/dp/000004458X
[1] Processing https://www.amazon.de/dp/000004458X
[2] Processing https://www.amazon.de/ref=cs_503_logo/
[3] Processing https://www.amazon.de/ref=cs_503_link/
[4] Processing https://www.amazon.de/gp/help/customer/display.html/ref=footer_cou/275-2496043-9483305?ie=UTF8&nodeId=505048
[5] Processing https://www.amazon.de/gp/help/customer/display.html/ref=footer_privacy?ie=UTF8&nodeId=3312401
api-services-support@amazon.com


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>